# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [29]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13153855725478548618
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 9884204317171019696
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [3]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
with open('data/small_vocab_fr', 'rt', encoding='utf-8') as f:
    french_sentences = f.read().split('\n')
    f.close()

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [4]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [5]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [20]:
english_words_counter.most_common()

[('is', 205858),
 (',', 140897),
 ('.', 129039),
 ('in', 75525),
 ('it', 75137),
 ('during', 74933),
 ('the', 67628),
 ('but', 63987),
 ('and', 59850),
 ('sometimes', 37746),
 ('usually', 37507),
 ('never', 37500),
 ('least', 27564),
 ('favorite', 27371),
 ('fruit', 27105),
 ('most', 14934),
 ('loved', 13666),
 ('liked', 13546),
 ('new', 12197),
 ('paris', 11334),
 ('india', 11277),
 ('united', 11270),
 ('states', 11270),
 ('california', 11250),
 ('jersey', 11225),
 ('france', 11170),
 ('china', 10953),
 ('he', 10786),
 ('she', 10786),
 ('grapefruit', 10118),
 ('your', 9734),
 ('my', 9700),
 ('his', 9700),
 ('her', 9700),
 ('fall', 9134),
 ('june', 9133),
 ('spring', 9102),
 ('january', 9090),
 ('winter', 9038),
 ('march', 9023),
 ('autumn', 9004),
 ('may', 8995),
 ('nice', 8984),
 ('september', 8958),
 ('july', 8956),
 ('april', 8954),
 ('november', 8951),
 ('summer', 8948),
 ('december', 8945),
 ('february', 8942),
 ('our', 8932),
 ('their', 8932),
 ('freezing', 8928),
 ('pleasant', 

For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [6]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(x)
    tokenized = tokenizer.texts_to_sequences(x)
    
    return tokenized, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, ',': 13, 'my': 14, 'study': 15, 'of': 16, 'lexicography': 17, 'won': 18, 'prize': 19, 'this': 20, 'is': 21, 'short': 22, 'sentence': 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 5, 6, 7, 8, 2, 9, 10, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 12, 13, 14, 3, 15, 16, 17, 18, 4, 19, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [20, 21, 4, 22, 23, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [7]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    l=0
    if length is None:
        for seq in x:
            current = len(seq)
            if current > l:
                l = current
        padded_x = pad_sequences(x, padding='post', maxlen=l)
    else:
        padded_x = pad_sequences(x, padding='post', maxlen=length)
    return padded_x
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

[[0, 1, 2, 3], [0, 1, 2, 3, 4, 5], [0, 1, 2]]
[[0 1 2 3 0 0]
 [0 1 2 3 4 5]
 [0 1 2 0 0 0]]
0
Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10  1]
  Output: [ 2  3  5  6  7  8  2  9 10  1  0  0]
Sequence 2 in x
  Input:  [11 12 13 14  3 15 16 17 18  4 19  1]
  Output: [11 12 13 14  3 15 16 17 18  4 19  1]
Sequence 3 in x
  Input:  [20 21  4 22 23  1]
  Output: [20 21  4 22 23  1  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [8]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 17
Max French sentence length: 23
English vocabulary size: 227
French vocabulary size: 354


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [9]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [10]:
# Creating {id: word} dictonary for english
english_index_to_word = {idx: word for word, idx in english_tokenizer.word_index.items()}
english_index_to_word[0] = '<PAD>'

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [14]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    
    # input shape is Sequence_legnth x 1
    input_ = Input(shape=input_shape[1:])
    
    # We're sampling the translation at each step, so output sequence length will 
    # be the same as the input sequence length. The returned hidden state will be a 
    # vector of length french_vocab_size, and the activation function will be softmax
    # since we're mapping the input to categories
    
    rnn_output = GRU(french_vocab_size, activation='softmax', return_sequences=True)(input_)
    model = Model(inputs=input_, outputs=rnn_output)
    
    # A single output of the rnn is expected to have length equals to french_vocab_size,
    # and the total output will be the sequence of word vectors, with length equals
    # to the input sequences length
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# The actual output of the rnn are vectors of length = french_vocab, 
# though sparse_categotical_crossentropy needs only a one dim label,
# which is the word_id
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 344)           357072    
Total params: 357,072
Trainable params: 357,072
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 23, 1)             0         
_________________________________________________________________
gru_5 (GRU)                  (None, 23, 354)           378072    
Total params: 378,072
Trainable params: 378,072
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
11

In [47]:
first_seq_index = 18
number_sequences = 5
model = simple_rnn_model
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    sentence = sentence.reshape((-1))
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("Sentence in english: \n", " ".join(sentence))
    print("Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))

Sentence in english: 
 paris is never freezing during november , but it is wonderful in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 inde inde jamais au au , , , il il <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is never rainy during january , but it is sometimes mild in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 inde est est est est est est , , il il il il <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 china is usually pleasant during november , and it is never quiet in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 inde inde jamais au au , , , il il il <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is never nice during february , but it is sometimes pleasant in april . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence

In [34]:
def simple_model_multiLayerRNN(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    dxzs
    # input shape is Sequence_legnth x 1
    input_ = Input(shape=input_shape[1:])

    # rnn_output
    rnn_output_1 = GRU(256, dropout=0.2, return_sequences=True)(input_)
    
    rnn_output_2 = GRU(256, dropout=0.2, return_sequences=True)(rnn_output_1)

    # Dense layer
    dense1 = TimeDistributed(Dense(500, activation='relu'))(rnn_output_2)
    
    # Dropout
    dropout = Dropout(0.2)(dense1)
    
    # Second Dense
    dense2 = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout)
    
    model = Model(inputs=input_, outputs=dense2)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model
# tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model_multiLayerRNN = simple_model_multiLayerRNN(tmp_x.shape,
                                                            max_french_sequence_length,
                                                            english_vocab_size,
                                                            french_vocab_size)

simple_rnn_model_multiLayerRNN.fit(tmp_x, 
                                   preproc_french_sentences,
                                   batch_size=1024,
                                   epochs=20,
                                   validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model_multiLayerRNN.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 23, 1)             0         
_________________________________________________________________
gru_6 (GRU)                  (None, 23, 256)           198144    
_________________________________________________________________
gru_7 (GRU)                  (None, 23, 256)           393984    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 23, 500)           128500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 500)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 23, 354)           177354    
Total params: 897,982
Trainable params: 897,982
Non-trainable params: 0
_________________________________________________________________
Trai

In [51]:
first_seq_index = 18
number_sequences = 5
model = simple_rnn_model_multiLayerRNN
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("\n Sentence in english: \n", " ".join(sentence))
    print("\n Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))


 Sentence in english: 
 paris is never freezing during november , but it is wonderful in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 new est parfois agréable en janvier , mais mais est agréable en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never rainy during january , but it is sometimes mild in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 les états-unis est généralement agréable en l' , l' automne est parfois parfois en en . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 china is usually pleasant during november , and it is never quiet in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 l' inde est parfois agréable en janvier , mais il est parfois parfois en juillet juillet <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never nice during february , but it is s

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [42]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # input shape sequences length x 1
    input_ = Input(shape=input_shape[1:])
    
    # Embedding
    embedding_dim = 300
    embed = Embedding(english_vocab_size, embedding_dim)(input_)
    
    # The input to the rnn layer shoud have shape (SequenceLength x vector )
    rnn_outputs = GRU(french_vocab_size, activation='softmax', return_sequences=True)(embed)
    
    model = Model(inputs=input_, outputs=rnn_outputs)
    
    learning_rate = 0.001
    
    # Compiling the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    
    return model

# Reshaping the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)

# Training the model
embed_rnn_model = embed_model(tmp_x.shape,
                              max_french_sequence_length, 
                              english_vocab_size,
                              french_vocab_size)
embed_rnn_model.fit(tmp_x,
                    preproc_french_sentences,
                    batch_size=1024,
                    epochs=20,
                    validation_split=0.2)

# Print prediction
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 23)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 23)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
gru_9 (GRU)                  (None, 23, 354)           695610    
Total params: 763,710
Trainable params: 763,710
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 20s - loss: 3.9649 - acc: 0.5797 - val_loss: nan - val_acc: 0.6100
Epoch 2/20
110288/110288 [==============================] - 20s - loss: 1.2559 - acc: 0.6262 - val_loss: nan - val_acc: 0.6350
Epoch 3/20
110288/110288 [==============================] - 20s - loss: 1.0623 - a

In [55]:
first_seq_index = 18
number_sequences = 5
model = embed_rnn_model
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("\n Sentence in english: \n", " ".join(sentence))
    print("\n Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))


 Sentence in english: 
 paris is never freezing during november , but it is wonderful in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 paris est jamais gel en novembre , , il est merveilleux merveilleux en . octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never rainy during january , but it is sometimes mild in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 les états-unis est glaciales jamais pluvieux au janvier , mais il glaciales parfois doux en octobre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 china is usually pleasant during november , and it is never quiet in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 chine est généralement agréable en novembre , et il est est jamais en en . . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never nice during february , but it is so

In [56]:
def embed_model_multiLayerRNN(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # input shape sequences length x 1
    input_ = Input(shape=input_shape[1:])
    print('Input shape should be (?, 23) and is: ', input_.shape)
    
    # Embedding
    embedding_dim = 300
    embed = Embedding(english_vocab_size, embedding_dim)(input_)
    print('Embedding layer shape should be (?, 23, 300), and is ', embed.shape)
    
    # GRU 1
    rnn_outputs_1 = GRU(256, dropout=0.2, return_sequences=True)(embed)
    print('RNN layer 1 outputs shape should be (?, 23, 512) and is: ', rnn_outputs_1.shape)
    
    # GRU 2
    rnn_outputs_2 = GRU(256, dropout=0.2, return_sequences=True)(rnn_outputs_1)
    print('RNN layer 2 outputs shape should be (?, 23, 512) and is: ', rnn_outputs_2.shape)
    
    
    # Dense
    dense1 = TimeDistributed(Dense(500, activation='relu'))(rnn_outputs_2)
    print('Dense1 layer shape should be (?, 23, 500) and is: ', dense1.shape)
    
    # Dropout
    dropout = Dropout(0.2)(dense1)
    
    # Dense2: output
    dense2 = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout)
    print('Dense2 layer shape should be (?, 23, 354) and is: ', dense2.shape)
    
    model = Model(inputs=input_, outputs=dense2)
    print('Model output shape: ', model.output_shape)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model
# tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print("Shape of tmp_x after padding: ", tmp_x.shape)

# TODO: Train the neural network
embed_rnn_multiLayerRNN = embed_model_multiLayerRNN(tmp_x.shape, 
                                                  max_french_sequence_length, 
                                                  english_vocab_size, 
                                                  french_vocab_size)
embed_rnn_multiLayerRNN.fit(tmp_x, 
                             preproc_french_sentences, 
                             batch_size = 1024,
                             epochs = 20,
                             validation_split = 0.2)

# Print prediction
print(logits_to_text(embed_rnn_multiLayerRNN.predict(tmp_x[:1])[0], french_tokenizer))

Shape of tmp_x after padding:  (137861, 23)
Input shape should be (?, 23) and is:  (?, 23)
Embedding layer shape should be (?, 23, 300), and is  (?, 23, 300)
RNN layer 1 outputs shape should be (?, 23, 512) and is:  (?, ?, 256)
RNN layer 2 outputs shape should be (?, 23, 512) and is:  (?, ?, 256)
Dense1 layer shape should be (?, 23, 500) and is:  (?, 23, 500)
Dense2 layer shape should be (?, 23, 354) and is:  (?, 23, 354)
Model output shape:  (None, 23, 354)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 23)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
gru_10 (GRU)                 (None, 23, 256)           427776    
_________________________________________________________________
gru_11 (G

In [57]:
first_seq_index = 0
number_sequences = 5
model = embed_rnn_multiLayerRNN
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("\n Sentence in english: \n", " ".join(sentence))
    print("\n Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))


 Sentence in english: 
 new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 new jersey est parfois calme en l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is usually chilly during july , and it is usually freezing in november . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 les états-unis est jamais froid en l' , mais il est parfois agréable décembre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 california is usually quiet during march , and it is usually hot in june . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 californie est généralement calme en mois de et il est généralement chaud en juin . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is sometimes mild during june , and it is cold in september . <

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [59]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_ = Input(shape=input_shape[1:])
    
    rnn_outputs = Bidirectional(GRU(french_vocab_size, activation='softmax', return_sequences=True))(input_)
    
    model = Model(inputs=input_, outputs=rnn_outputs)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model
    
# tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# instantiatin model
bd_rnn = bd_model(tmp_x.shape, 
                  max_french_sequence_length,
                  english_vocab_size,
                  french_vocab_size)

# fitting model to data
bd_rnn.fit(tmp_x,
           preproc_french_sentences,
           batch_size=2014,
           epochs=20,
           validation_split=0.2)

# Print prediction
print(logits_to_text(bd_rnn.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 23, 1)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 23, 708)           756144    
Total params: 756,144
Trainable params: 756,144
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 31s - loss: 6.4110 - acc: 0.0307 - val_loss: 6.0241 - val_acc: 0.0325
Epoch 2/20
110288/110288 [==============================] - 31s - loss: 5.8350 - acc: 0.0571 - val_loss: 5.6623 - val_acc: 0.1537
Epoch 3/20
110288/110288 [==============================] - 31s - loss: 5.3885 - acc: 0.3443 - val_loss: 5.2198 - val_acc: 0.3822
Epoch 4/20
110288/110288 [==============================] - 31s - loss: 5.1153 - acc: 0.381

In [62]:
first_seq_index = 0
number_sequences = 5
model = bd_rnn
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("\n Sentence in english: \n", " ".join(sentence))
    print("\n Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))


 Sentence in english: 
 new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 est est est est <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is usually chilly during july , and it is usually freezing in november . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 inde est est est est <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 california is usually quiet during march , and it is usually hot in june . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 est est est <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is sometimes mild during june , and it is cold i

In [63]:
def bd_model_multiRNN(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_ = Input(shape=input_shape[1:])
    
    # Embedding
    embed_dim = 300
    embed = Embedding(english_vocab_size, embed_dim)(input_)
    
    # GRU 1
    rnn_outputs_1 = Bidirectional(GRU(256, dropout=0.2, return_sequences=True))(embed)
    
    # GRU 2
    rnn_outputs_2 = Bidirectional(GRU(256, dropout=0.2, return_sequences=True))(rnn_outputs_1)
    
    # Dense layer 1
    dense1 = TimeDistributed(Dense(500, activation='relu'))(rnn_outputs_2)
    
    # Dropout
    dropout = Dropout(0.2)(dense1)
    
    # Dense layer 2: output
    dense2 = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout)
    
    # Model
    model = Model(inputs=input_, outputs=dense2)
    
    learning_rate = 0.001
    
    # Compiling model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model
    
# tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

bd_rnn_multiRNN = bd_model_multiRNN(tmp_x.shape, 
                  max_french_sequence_length,
                  english_vocab_size,
                  french_vocab_size)

bd_rnn_multiRNN.fit(tmp_x,
                    preproc_french_sentences,
                    batch_size=2014,
                    epochs=20,
                    validation_split=0.2)

# Print prediction
print(logits_to_text(bd_rnn_multiRNN.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 23)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 23, 512)           855552    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 23, 512)           1181184   
_________________________________________________________________
time_distributed_5 (TimeDist (None, 23, 500)           256500    
_________________________________________________________________
dropout_3 (Dropout)          (None, 23, 500)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 23, 354)           177354    
Total para

In [64]:
first_seq_index = 0
number_sequences = 10
model = bd_rnn_multiRNN
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("Sentence in english: \n", " ".join(sentence))
    print("Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))

Sentence in english: 
 new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is usually chilly during july , and it is usually freezing in november . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 les états-unis est généralement froid en juillet , et il gèle habituellement en novembre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 california is usually quiet during march , and it is usually hot in june . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 california est généralement calme en mars , et il est généralement chaud en juin . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is sometimes mild during june , and it is cold in september . 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [39]:
from keras.layers import GRU, LSTM, Dense

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    encoder_inputs = Input(shape=input_shape[1:])
    print('Encoder inputs shape: ', encoder_inputs.shape)
    
    # Embedding
    embed_dim = 300
    embed = Embedding(english_vocab_size, embed_dim)
    embed_outputs = embed(encoder_inputs)
    print('Embed output shape: ', embed_outputs.shape)
    # Encoder
    encoder_dim = 256
    encoder = LSTM(encoder_dim, return_state=True)
    encoder_out, state_h, state_c = encoder(embed_outputs)
    encoder_states = [state_h, state_c]
    print('Encoder outputs and states shape: ', encoder_out.shape, state_h.shape)
    
    # Decoder
    decoder_dim = 256
    decoder_inputs = Input(shape=(None,))
    decoder_embed = Embedding(french_vocab_size, embed_dim)
    dec_embed_outputs = decoder_embed(decoder_inputs)
    
    decoder = LSTM(256, dropout=0.2, return_sequences=True)
    decoder_outputs = decoder(dec_embed_outputs, initial_state=encoder_states)
    
    decoder_dense = Dense(french_vocab_size, activation='softmax')
    dec_dense_outputs = decoder_dense(decoder_outputs)

    # create model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=dec_dense_outputs)
    
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary
    return model
# tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

encoder_decoder = encdec_model(tmp_x.shape,
                               max_french_sequence_length,
                               english_vocab_size,
                               french_vocab_size)
print(type(encoder_decoder))

encoder_decoder.fit(tmp_x, 
                    preproc_french_sentences,
                    batch_size=1024,
                    epochs=20,
                    validation_split=0.2)

Encoder inputs shape:  (?, 23)
Embed output shape:  (?, 23, 300)
Encoder outputs and states shape:  (?, 256) (?, 256)
<class 'keras.engine.training.Model'>


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[19, 25,  1, ...,  0,  0,  0],
       [ 7, 22, 23, ...,  0,  0,  0],
       [24,  1, 11, ...,  0,  0,  0],
       ...,
       [26,  1, 12, ...,  0,  0,  0],
       [ 7, 93,  1, ...,  0,  0,  0...

In [ ]:
# using TENSORFLOW to create the encoder decoder model
import tensorflow as tf



### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [66]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_ = Input(shape=input_shape[1:])
    
    # Embedding
    embed_dim = 300
    embed = Embedding(english_vocab_size, embed_dim)(input_)
    
    # GRU 1
    rnn_outputs_1 = Bidirectional(GRU(512, dropout=0.2, return_sequences=True))(embed)
    
    # GRU 2
    rnn_outputs_2 = Bidirectional(GRU(512, dropout=0.2, return_sequences=True))(rnn_outputs_1)
    
    # Dense layer 1
    dense1 = TimeDistributed(Dense(500, activation='relu'))(rnn_outputs_2)
    
    # Dropout
    dropout = Dropout(0.2)(dense1)
    
    # Dense layer 2: output
    dense2 = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout)
    
    # Model
    model = Model(inputs=input_, outputs=dense2)
    
    learning_rate = 0.001
    
    # Compiling model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    
    return model


print('Final Model Loaded')
# TODO: Train the final model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

final_rnn = model_final(tmp_x.shape, 
                  max_french_sequence_length,
                  english_vocab_size,
                  french_vocab_size)

final_rnn.fit(tmp_x,
              preproc_french_sentences,
              batch_size=2014,
              epochs=20,
              validation_split=0.2)


# tests.test_model_final(model_final)

Final Model Loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 23)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 23, 1024)          2497536   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 1024)          4721664   
_________________________________________________________________
time_distributed_9 (TimeDist (None, 23, 500)           512500    
_________________________________________________________________
dropout_5 (Dropout)          (None, 23, 500)           0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 23, 354)           17

In [68]:
first_seq_index = 18
number_sequences = 5
model = final_rnn
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("Sentence in english: \n", " ".join(sentence))
    print("Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))

Sentence in english: 
 paris is never freezing during november , but it is wonderful in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 paris est jamais le gel en novembre , mais il est merveilleux en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is never rainy during january , but it is sometimes mild in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 les états-unis est jamais pluvieux en janvier , mais il est parfois doux en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 china is usually pleasant during november , and it is never quiet in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in French: 
 chine est généralement agréable en novembre , et il est jamais tranquille en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Sentence in english: 
 the united states is never nice during february , but it is sometimes pleasa

## Prediction (IMPLEMENTATION)

In [69]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    # padding english sentences so they have same length as french
    tmp_x = pad(x, y.shape[1])
    x_vocab_size = len(x_tk.word_index)
    y_vocab_size = len(y_tk.word_index)

    model = model_final(tmp_x.shape,
                        y.shape[1],
                        x_vocab_size,
                        y_vocab_size)
    
    model.fit(tmp_x,
              y,
              batch_size=1024,
              epochs=20,
              validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=y.shape[1], padding='post')
    sentences = np.array([sentence[0], tmp_x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

    return model

final_pred = final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 23)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 23, 1024)          2497536   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 23, 1024)          4721664   
_________________________________________________________________
time_distributed_11 (TimeDis (None, 23, 500)           512500    
_________________________________________________________________
dropout_6 (Dropout)          (None, 23, 500)           0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 23, 354)           177354    
Total para

In [70]:
# using LSTM instead of GRU
from keras.layers import LSTM
def LSTM_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    input_ = Input(shape=input_shape[1:])
    
    # Embedding
    embed_dim = 300
    embed = Embedding(english_vocab_size, embed_dim)(input_)
    
    # Bidirectional LSTM
    rnn_outputs_1 = Bidirectional(LSTM(256, dropout=0.2, return_sequences=True))(embed)
    rnn_outputs_2 = Bidirectional(LSTM(256, dropout=0.2, return_sequences=True))(rnn_outputs_1)
    
    # Dense layers
    dense1 = TimeDistributed(Dense(500, activation='relu'))(rnn_outputs_2)
    drop = Dropout(0.2)(dense1)
    dense2 = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(drop)
    
    # Model
    model = Model(inputs=input_, outputs=dense2)
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    model.summary()
    
    return model

lstm_rnn = LSTM_model(tmp_x.shape,
                      max_french_sequence_length,
                      english_vocab_size,
                      french_vocab_size)

lstm_rnn.fit(tmp_x,
             preproc_french_sentences,
             batch_size=1024,
             epochs=20,
             validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 23)                0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 23, 300)           68100     
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 23, 512)           1140736   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 23, 512)           1574912   
_________________________________________________________________
time_distributed_13 (TimeDis (None, 23, 500)           256500    
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 500)           0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 23, 354)           177354    
Total para

In [72]:
first_seq_index = 18
number_sequences = 5
model = lstm_rnn
for i in range(first_seq_index, first_seq_index+number_sequences):
    translation_logits = model.predict(tmp_x[i:i+1])
    sentence = tmp_x[i]
    try:
        sentence = sentence.reshape((-1))
    except:
        pass
    sentence = [english_index_to_word[idx] for idx in sentence]
    print("\n Sentence in english: \n", " ".join(sentence))
    print("\n Sentence in French: \n", logits_to_text(translation_logits[0], french_tokenizer))


 Sentence in english: 
 paris is never freezing during november , but it is wonderful in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 paris est jamais le gel en novembre , mais il est merveilleux en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never rainy during january , but it is sometimes mild in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 les états-unis est jamais pluvieux en janvier , mais il est parfois doux en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 china is usually pleasant during november , and it is never quiet in october . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in French: 
 chine est généralement agréable en novembre , et il est jamais tranquille en octobre . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

 Sentence in english: 
 the united states is never nice during february , but it is so

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [15]:
# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x_train, tmp_x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=23)

In [16]:
print(len(tmp_x_train), len(tmp_x_test), len(y_train), len(y_test))

110288 27573 110288 27573


In [ ]:
final_with_split = model_final(tmp_x_train.shape,
                               max_french_sequence_length,
                               english_vocab_size,
                               french_vocab_size)
final_with_split.fit(tmp_x_train, 
                     y_train,
                     batch_size=2014,
                     epochs=20,
                     validation_split=0.2)

In [ ]:
final_with_split.evaluate(tmp_x_test, y_test, batch_size=2014)

In [ ]:
# Testing model on the test set
final_with_split.test_on_batch(tmp_x_test, y_test)